In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# 数据处理

def sparseFeature(feat, feat_num, embed_dim=4):
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def denseFeature(feat):
    return {'feat': feat}


def create_criteo_dataset(file, embed_dim=8, read_part=True, sample_num=100000, test_size=0.2):
    names = ['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11',
             'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
             'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22',
             'C23', 'C24', 'C25', 'C26']

    if read_part:
        data_df = pd.read_csv(file, sep='\t', iterator=True, header=None,
                          names=names)
        data_df = data_df.get_chunk(sample_num)

    else:
        data_df = pd.read_csv(file, sep='\t', header=None, names=names)

    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]

    data_df[sparse_features] = data_df[sparse_features].fillna('-1')
    data_df[dense_features] = data_df[dense_features].fillna(0)

    for feat in sparse_features:
        le = LabelEncoder()
        data_df[feat] = le.fit_transform(data_df[feat])

    dense_features = [feat for feat in data_df.columns if feat not in sparse_features + ['label']]

    mms = MinMaxScaler(feature_range=(0, 1))
    data_df[dense_features] = mms.fit_transform(data_df[dense_features])

    feature_columns = [[denseFeature(feat) for feat in dense_features]] + \
                      [[sparseFeature(feat, len(data_df[feat].unique()), embed_dim=embed_dim)
                        for feat in sparse_features]]

    train, test = train_test_split(data_df, test_size=test_size)

    train_X = [train[dense_features].values.astype('float32'), train[sparse_features].values.astype('int32')]
    train_y = train['label'].values.astype('int32')
    test_X = [test[dense_features].values.astype('float32'), test[sparse_features].values.astype('int32')]
    test_y = test['label'].values.astype('int32')

    return feature_columns, (train_X, train_y), (test_X, test_y)

In [3]:
# 建立数据集
file = './dataset/Criteo/train.txt'
read_part = True
sample_num = 5000000
test_size = 0.2

embed_dim = 8

feature_columns, train, test = create_criteo_dataset(file=file,
                                                     embed_dim=embed_dim,
                                                     read_part=read_part,
                                                     sample_num=sample_num,
                                                     test_size=test_size)
train_X, train_y = train
test_X, test_y = test

In [4]:
# LR模型
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding, Dense


class LR(keras.Model):
    def __init__(self, feature_columns, embed_reg=1e-4):
        super(LR, self).__init__()
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.embed_layers = {
            'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
                                         input_length=1,
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=l2(embed_reg))
            for i, feat in enumerate(self.sparse_feature_columns)
        }
        self.dense_final = Dense(1)

    def call(self, inputs):
        dense_inputs, sparse_inputs = inputs
        sparse_embed = tf.concat([self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i])
                                  for i in range(sparse_inputs.shape[1])], axis=-1)
        x = tf.concat([sparse_embed, dense_inputs], axis=-1)

        outputs = tf.nn.sigmoid(self.dense_final(x))
        return outputs

    def summary(self):
        dense_inputs = Input(shape=(len(self.dense_feature_columns),), dtype=tf.float32)
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        keras.Model(inputs=[dense_inputs, sparse_inputs],
                    outputs=self.call([dense_inputs, sparse_inputs])).summary()


In [5]:
# 超参数
learning_rate = 0.001
batch_size = 4096
epochs = 10

# 建立LR模型（使用双卡）
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = LR(feature_columns)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None,)]            0           input_2[0][0]                    
______________________________________________________________________________________________

In [6]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 28 all-reduces with algorithm = nccl, num_packs = 1, agg_small

In [7]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4734
test AUC 0.7662


In [8]:
# FM模型
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.regularizers import l2

class FM(tf.keras.Model):
    def __init__(self, feature_columns, embed_reg=1e-4):
        super(FM, self).__init__()
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.embed_reg = embed_reg
        self.f_dense = Dense(units=1, activation=None)
        self.embed_layers = {
            'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
                                         input_length=1,
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=l2(self.embed_reg))
            for i, feat in enumerate(self.sparse_feature_columns)
        }

    def call(self, inputs, **kwargs):

        dense_inputs, sparse_inputs = inputs

        # Embedding 层
        sparse_embed = [self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i]) for i in
                        range(sparse_inputs.shape[1])]
        embed = tf.transpose(tf.convert_to_tensor(sparse_embed), perm=[1, 0, 2])

        concated_embeds_value = embed

        square_of_sum = tf.square(tf.reduce_sum(
            concated_embeds_value, axis=1, keepdims=True))
        sum_of_square = tf.reduce_sum(
            concated_embeds_value * concated_embeds_value, axis=1, keepdims=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * tf.reduce_sum(cross_term, axis=2, keepdims=False)

        # 一阶
        sparse_embed = tf.concat(sparse_embed, axis=-1)
        stack = tf.concat([dense_inputs, sparse_embed], axis=-1)
        first_order_outputs = self.f_dense(stack)

        # 输出层
        outputs = tf.nn.sigmoid(tf.add(first_order_outputs, cross_term))

        return outputs

    def summary(self, **kwargs):
        dense_inputs = tf.keras.Input(shape=(len(self.dense_feature_columns),), dtype=tf.float32)
        sparse_inputs = tf.keras.Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        tf.keras.Model(inputs=[dense_inputs, sparse_inputs], outputs=self.call([dense_inputs, sparse_inputs])).summary()


In [9]:
# 超参数
learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = FM(feature_columns=feature_columns, embed_reg=1e-4)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_26 (T [(None,)]            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_27 (T [(None,)]            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_28 (T [(None,)]            0           input_4[0][0]                    
____________________________________________________________________________________________

In [10]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 28 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 28 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 183s 57us/sample - loss: 0.5865 - auc_1: 0.7278 - val_loss: 0.5061 - val_auc_1: 0.7539
Epoch 2/10
3200000/3200000 [==============================] - 174s 54us/sample - loss: 0.5031 - auc_1: 0.7597 - val_loss: 0.5037 - val_auc_1: 0.7609
Epoch 3/10
3200000/3200000 [==============================] - 173s 54us/sample - loss: 0.4998 - auc_1: 0.7657 - val_loss: 0.5006 - val_auc_1: 0.7654
Epoch 4/10
3200000/3200000 [==============================] - 174s 55us/sample - loss: 0.4970 - auc_1: 0.7697 - val_loss: 0.4984 - val_auc_1: 0.7678
Epoch 5/10
3200000/3200000 [===================

In [11]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4681
test AUC 0.7739


In [12]:
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding, Dense, Dropout, Input


class AFM(keras.Model):
    def __init__(self, feature_columns, att_vector=8, activation='relu', dropout=0.5, embed_reg=1e-4):
        super(AFM, self).__init__()
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.embed_layers = {
            'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
                                         input_length=1,
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=l2(embed_reg))
            for i, feat in enumerate(self.sparse_feature_columns)
        }
        self.attention_W = Dense(units=att_vector, activation=activation, use_bias=True)
        self.attention_dense = Dense(units=1, activation=None)
        self.dropout = Dropout(dropout)
        self.dense = Dense(units=1, activation=None)
        self.f_dense = Dense(units=1, activation=None)

    def call(self, inputs):
        # 输入层
        dense_inputs, sparse_inputs = inputs
        # Embedding 层
        sparse_embed = [self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i]) for i in range(sparse_inputs.shape[1])]
        embed = tf.transpose(tf.convert_to_tensor(sparse_embed), perm=[1, 0, 2])  # (None, len(sparse_inputs), embed_dim)
        # Pair-wise Interaction 层
        row = []
        col = []
        for r, c in itertools.combinations(range(len(self.sparse_feature_columns)), 2):
            row.append(r)
            col.append(c)
        p = tf.gather(embed, row, axis=1)  # (None, (len(sparse) * len(sparse) - 1) / 2, k)
        q = tf.gather(embed, col, axis=1)  # (None, (len(sparse) * len(sparse) - 1) / 2, k)

        p_attention_output = self.attention(p)  # (None, k)
        q_attention_output = self.attention(q)  # (None, k)
        attention_output = p_attention_output * q_attention_output  # (None, (len(sparse) * len(sparse) - 1) / 2, k)

        attention_output = self.dense(attention_output)
            
        # LR一阶
        sparse_embed = tf.concat(sparse_embed, axis=-1)
        stack = tf.concat([dense_inputs, sparse_embed], axis=-1)
        first_order_outputs = self.f_dense(stack)    
        
        
        # 输出层
        outputs = tf.nn.sigmoid(tf.add(first_order_outputs, attention_output))

        return outputs

    def summary(self):
        dense_inputs = Input(shape=(len(self.dense_feature_columns),), dtype=tf.float32)
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        keras.Model(inputs=[dense_inputs, sparse_inputs], outputs=self.call([dense_inputs, sparse_inputs])).summary()

    def attention(self, bi_interaction):
        a = self.attention_W(bi_interaction)  # (None, (len(sparse) * len(sparse) - 1) / 2, t)
        a = self.attention_dense(a)  # (None, (len(sparse) * len(sparse) - 1) / 2, 1)
        self.a_score = tf.nn.softmax(a, axis=1)  # (None, (len(sparse) * len(sparse) - 1) / 2, 1)
        outputs = tf.reduce_sum(bi_interaction * self.a_score, axis=1)  # (None, embed_dim)
        return outputs

In [13]:
# 超参数
att_vector = 8
dropout = 0.5
activation = 'relu'
embed_reg = 1e-5

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = AFM(feature_columns, att_vector, activation, dropout, embed_reg)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_52 (T [(None,)]            0           input_6[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_53 (T [(None,)]            0           input_6[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_54 (T [(None,)]            0           input_6[0][0]                    
____________________________________________________________________________________________

In [14]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 34 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 34 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 215s 67us/sample - loss: 0.5215 - auc_2: 0.7269 - val_loss: 0.4934 - val_auc_2: 0.7564
Epoch 2/10
3200000/3200000 [==============================] - 205s 64us/sample - loss: 0.4907 - auc_2: 0.7635 - val_loss: 0.4923 - val_auc_2: 0.7621
Epoch 3/10
3200000/3200000 [==============================] - 205s 64us/sample - loss: 0.4886 - auc_2: 0.7693 - val_loss: 0.4910 - val_auc_2: 0.7650
Epoch 4/10
3200000/3200000 [==============================] - 206s 64us/sample - loss: 0.4864 - auc_2: 0.7737 - val_loss: 0.4905 - val_auc_2: 0.7665
Epoch 5/10
3200000/3200000 [===================

In [15]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4716
test AUC 0.7679


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding, Dense, Layer, Dropout


class CrossFM(keras.Model):
    def __init__(self, feature_columns, layer_num,
                 dnn_dropout=0., embed_reg=1e-4, cross_w_reg=1e-4, cross_b_reg=1e-4):
        """
        CrossFM 
        :param feature_columns: A list. dense_feature_columns + sparse_feature_columns
        :param dnn_dropout: A scalar. Dropout of dnn.
        :param embed_reg: A scalar. The regularizer of embedding.
        :param cross_w_reg: A scalar. The regularizer of cross network.
        :param cross_b_reg: A scalar. The regularizer of cross network.
        """
        super(CrossFM, self).__init__()
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.layer_num = layer_num
        self.reg_w = cross_w_reg
        self.reg_b = cross_b_reg
        self.embed_layers = {
            'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
                                         input_length=1,
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=l2(embed_reg))
            for i, feat in enumerate(self.sparse_feature_columns)
        }
        self.cross = Cross(self.layer_num, cross_w_reg, cross_b_reg)
        self.first_order_dense = Dense(1)
        self.dense_final = Dense(1)

    def call(self, inputs):
        dense_inputs, sparse_inputs = inputs
        sparse_embed = tf.concat([self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i])
                                  for i in range(sparse_inputs.shape[1])], axis=-1)
        x = tf.concat([sparse_embed, dense_inputs], axis=-1)

        order_x = self.cross(x)

        order_x.append(x)

        total_x = tf.concat(order_x, axis=-1)
        outputs = tf.nn.sigmoid(self.dense_final(total_x))
        return outputs

    def summary(self):
        dense_inputs = Input(shape=(len(self.dense_feature_columns),), dtype=tf.float32)
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        keras.Model(inputs=[dense_inputs, sparse_inputs],
                    outputs=self.call([dense_inputs, sparse_inputs])).summary()

class Cross(Layer):
    """
    Cross Network
    """

    def __init__(self, layer_num, reg_w=1e-4, reg_b=1e-4):
        """
        :param layer_num: A scalar. The depth of cross network
        :param reg_w: A scalar. The regularizer of w
        :param reg_b: A scalar. The regularizer of b
        """
        super(Cross, self).__init__()
        self.layer_num = layer_num
        self.reg_w = reg_w
        self.reg_b = reg_b

    def build(self, input_shape):
        dim = int(input_shape[-1])
        self.cross_weights = [
            self.add_weight(name='w_' + str(i),
                            shape=(dim, 1),
                            initializer='random_uniform',
                            regularizer=l2(self.reg_w),
                            trainable=True
                            )
            for i in range(self.layer_num)]
        self.cross_bias = [
            self.add_weight(name='b_' + str(i),
                            shape=(dim, 1),
                            initializer='random_uniform',
                            regularizer=l2(self.reg_b),
                            trainable=True
                            )
            for i in range(self.layer_num)]

    def call(self, inputs, **kwargs):
        order_x = []
        x_0 = tf.expand_dims(inputs, axis=2)  # (None, dim, 1)
        x_l = x_0  # (None, dim, 1)
        for i in range(self.layer_num):
            x_l1 = tf.tensordot(x_l, self.cross_weights[i], axes=[1, 0])  # (None, dim, dim)
            x_l = tf.matmul(x_0, x_l1) + self.cross_bias[i] + x_l  # (None, dim, 1)
            x_l_tmp = tf.squeeze(x_l, axis=2)  # (None, dim)
            order_x.append(x_l_tmp)
        return order_x

In [17]:
# 超参数
att_vector = 8
dropout = 0.5
layer_num = 1

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = CrossFM(feature_columns, layer_num, dnn_dropout=dropout)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_78 (T [(None,)]            0           input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_79 (T [(None,)]            0           input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_80 (T [(None,)]            0           input_8[0][0]                    
____________________________________________________________________________________________

In [18]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 30 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 30 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 183s 57us/sample - loss: 0.5837 - auc_3: 0.7308 - val_loss: 0.5036 - val_auc_3: 0.7570
Epoch 2/10
3200000/3200000 [==============================] - 178s 56us/sample - loss: 0.4996 - auc_3: 0.7622 - val_loss: 0.4989 - val_auc_3: 0.7629
Epoch 3/10
3200000/3200000 [==============================] - 177s 55us/sample - loss: 0.4947 - auc_3: 0.7670 - val_loss: 0.4941 - val_auc_3: 0.7663
Epoch 4/10
3200000/3200000 [==============================] - 177s 55us/sample - loss: 0.4906 - auc_3: 0.7708 - val_loss: 0.4910 - val_auc_3: 0.7687
Epoch 5/10
3200000/3200000 [===================

In [19]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4706
test AUC 0.77


In [20]:
# 超参数
att_vector = 8
dropout = 0.5
layer_num = 2

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = CrossFM(feature_columns, layer_num, dnn_dropout=dropout)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_104 ( [(None,)]            0           input_10[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_105 ( [(None,)]            0           input_10[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_106 ( [(None,)]            0           input_10[0][0]                   
____________________________________________________________________________________________

In [21]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 191s 60us/sample - loss: 0.5781 - auc_4: 0.7414 - val_loss: 0.4980 - val_auc_4: 0.7633
Epoch 2/10
3200000/3200000 [==============================] - 184s 58us/sample - loss: 0.4936 - auc_4: 0.7685 - val_loss: 0.4928 - val_auc_4: 0.7688
Epoch 3/10
3200000/3200000 [==============================] - 183s 57us/sample - loss: 0.4893 - auc_4: 0.7728 - val_loss: 0.4897 - val_auc_4: 0.7707
Epoch 4/10
3200000/3200000 [==============================] - 184s 58us/sample - loss: 0.4870 - auc_4: 0.7761 - val_loss: 0.4881 - val_auc_4: 0.7726
Epoch 5/10
3200000/3200000 [===================

In [22]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.468
test AUC 0.7736


In [5]:
# 超参数
att_vector = 8
dropout = 0.5
layer_num = 3

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = CrossFM(feature_columns, layer_num, dnn_dropout=dropout)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None,)]            0           input_2[0][0]                    
______________________________________________________________________________________________

In [6]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 34 all-reduces with algorithm = nccl, num_packs = 1, agg_small

In [7]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4671
test AUC 0.7751


In [8]:
# 超参数
att_vector = 8
dropout = 0.5
layer_num = 4

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = CrossFM(feature_columns, layer_num, dnn_dropout=dropout)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_26 (T [(None,)]            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_27 (T [(None,)]            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_28 (T [(None,)]            0           input_4[0][0]                    
____________________________________________________________________________________________

In [9]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 201s 63us/sample - loss: 0.5726 - auc_1: 0.7484 - val_loss: 0.4935 - val_auc_1: 0.7679
Epoch 2/10
3200000/3200000 [==============================] - 194s 60us/sample - loss: 0.4896 - auc_1: 0.7718 - val_loss: 0.4886 - val_auc_1: 0.7731
Epoch 3/10
3200000/3200000 [==============================] - 194s 60us/sample - loss: 0.4888 - auc_1: 0.7792 - val_loss: 0.4893 - val_auc_1: 0.7752
Epoch 4/10
3200000/3200000 [==============================] - 194s 61us/sample - loss: 0.4908 - auc_1: 0.7950 - val_loss: 0.5054 - val_auc_1: 0.7738


In [10]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.4697
test AUC 0.7727


In [11]:
# 超参数
att_vector = 8
dropout = 0.5
layer_num = 5

learning_rate = 0.001
batch_size = 4096
epochs = 10

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = CrossFM(feature_columns, layer_num, dnn_dropout=dropout)
    model.summary()
    # 模型编译
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),metrics=[AUC()])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_52 (T [(None,)]            0           input_6[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_53 (T [(None,)]            0           input_6[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_54 (T [(None,)]            0           input_6[0][0]                    
____________________________________________________________________________________________

In [12]:
# 训练
model.fit(
        train_X,
        train_y,
        epochs=epochs,
        # 防止过拟合 
        callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        batch_size=batch_size,
        validation_split=0.2
    )

Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 38 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 38 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3200000/3200000 [==============================] - 205s 64us/sample - loss: 0.5721 - auc_2: 0.7505 - val_loss: 0.4910 - val_auc_2: 0.7692
Epoch 2/10
3200000/3200000 [==============================] - 190s 59us/sample - loss: 0.4896 - auc_2: 0.7723 - val_loss: 0.4874 - val_auc_2: 0.7727
Epoch 3/10
3200000/3200000 [==============================] - 199s 62us/sample - loss: 0.4884 - auc_2: 0.7801 - val_loss: 0.4885 - val_auc_2: 0.7772
Epoch 4/10
3200000/3200000 [==============================] - 198s 62us/sample - loss: 0.4915 - auc_2: 0.7970 - val_loss: 0.5077 - val_auc_2: 0.7745


In [13]:
# 测试
from sklearn.metrics import log_loss, roc_auc_score

pred_ans = model.predict(test_X, batch_size=batch_size)
print("test LogLoss", round(log_loss(test_y, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_y, pred_ans), 4))

test LogLoss 0.469
test AUC 0.7725
